In [1]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🔥 Using GPU: {gpu_name}")
    torch.backends.cudnn.benchmark = True  # 입력 크기 고정이면 더 빠름
else:
    device = torch.device("cpu")
    print("⚠️ GPU not available, using CPU.")

🔥 Using GPU: NVIDIA GeForce RTX 3090


In [2]:
import os, random, numpy as np, torch

# ------------------------
# Reproducibility & GPU Optimization
# ------------------------
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# ⚙️ cuDNN settings
torch.backends.cudnn.deterministic = False  # True면 재현성↑, 속도↓
torch.backends.cudnn.benchmark = True       # 입력 크기 고정이면 속도↑
torch.backends.cudnn.enabled = True

# ⚡ Device check
if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🔥 Using GPU: {gpu_name}")
else:
    device = torch.device("cpu")
    print("⚠️ Using CPU. (No CUDA detected)")


🔥 Using GPU: NVIDIA GeForce RTX 3090


In [3]:
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import pandas as pd
import os

class ImageDataset(Dataset):
    def __init__(self, csv_or_df, path, transform=None):
        # csv 경로(str) or DataFrame 모두 허용
        if isinstance(csv_or_df, str):
            self.df = pd.read_csv(csv_or_df).values
        else:
            self.df = csv_or_df.values

        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)).convert("RGB"))
        if self.transform:
            img = self.transform(image=img)["image"]
        return img, target



In [4]:
from tqdm import tqdm
import torch
from sklearn.metrics import accuracy_score, f1_score

def train_one_epoch(loader, model, optimizer, loss_fn, device, scaler=None):
    model.train()
    train_loss = 0.0
    preds_list, targets_list = [], []

    pbar = tqdm(loader, desc="Training", leave=False)

    for images, targets in pbar:
        images, targets = images.to(device, non_blocking=True), targets.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        # ⚡ Mixed Precision Training
        with torch.cuda.amp.autocast(enabled=(scaler is not None)):
            preds = model(images)
            loss = loss_fn(preds, targets)

        # ✅ backward (scaler 있을 때와 없을 때 모두 지원)
        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        # Logging
        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())
        pbar.set_postfix(loss=f"{loss.item():.4f}")

    # Epoch metrics
    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }


In [5]:
# ------------------------
# ⚡ Device setting
# ------------------------
if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🔥 Using GPU: {gpu_name}")
    torch.backends.cudnn.benchmark = True  # 입력 크기 고정이면 속도 ↑
else:
    device = torch.device("cpu")
    print("⚠️ Using CPU. (No CUDA detected)")

# ------------------------
# ⚙️ Data config
# ------------------------
data_path = "../../../data/raw/"

# ------------------------
# 🧠 Model config
# ------------------------
# 문서 분류 정확도 우선 세팅 (ConvNeXt-Tiny)
model_name = "convnext_tiny.fb_in22k_ft_in1k"  # ImageNet22k → 1k fine-tuned 가중치 사용
IMG_CHANNELS = 3

# ------------------------
# 📈 Training config (RTX 3090 최적화)
# ------------------------
img_size = 384            # 문서 이미지면 384~512 추천
LR = 5e-4                 # ConvNeXt-Tiny에도 잘 맞는 값
EPOCHS = 15               # 약간 더 긴 학습 추천 (10 → 15)
BATCH_SIZE = 64           # ConvNeXt는 EfficientNet보다 메모리 조금 더 씀
num_workers = 8           # CPU 병렬 처리 강화
pin_memory = True
persistent_workers = True
prefetch_factor = 4       # DataLoader 성능 개선

# ------------------------
# 💡 Notes
# ------------------------
# - ConvNeXt-Tiny는 구조적으로 EfficientNet보다 깊고 넓은 모델이라,
#   GPU 사용량은 약 1.5~2배 높지만, 문서(텍스트/도장/레이아웃) 분류 성능이 훨씬 좋음.
# - AMP 사용 시 속도는 거의 동일, 메모리 절약 효과 큼.
# - 모델 로드는 아래처럼 하면 됨:
#     import timm
#     model = timm.create_model(model_name, pretrained=True, num_classes=NUM_CLASSES)
#     model.to(device)


🔥 Using GPU: NVIDIA GeForce RTX 3090


In [6]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# ------------------------
# 🧩 Train Transform
# ------------------------
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),        # 해상도 통일
    A.HorizontalFlip(p=0.5),                          # 문서 좌우 뒤집기 허용
    A.ShiftScaleRotate(
        shift_limit=0.02, scale_limit=0.05, rotate_limit=3, p=0.3
    ),                                                # 문서 기울기 약간만 허용
    A.OneOf([
        A.MotionBlur(blur_limit=3, p=0.5),
        A.GaussNoise(var_limit=(5, 30), p=0.5),
    ], p=0.2),                                        # 스캔 노이즈·흐림 약하게
    A.RandomBrightnessContrast(
        brightness_limit=0.1, contrast_limit=0.1, p=0.2
    ),
    A.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    ),
    ToTensorV2(),
])

# ------------------------
# 🧩 Validation / Test Transform
# ------------------------
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    ),
    ToTensorV2(),
])


/data/ephemeral/home/py310/lib/python3.10/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_42617/4228722789.py:15: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5, 30), p=0.5),


In [7]:
from sklearn.model_selection import train_test_split

# 1️⃣ train.csv 로드
full_df = pd.read_csv("../../../data/raw/train.csv")

# 2️⃣ train/val 분리 (stratify로 클래스 균형 유지)
trn_df, val_df = train_test_split(
    full_df, test_size=0.2, stratify=full_df['target'], random_state=42
)

# 3️⃣ Dataset 정의
trn_dataset = ImageDataset(trn_df, "../../../data/raw/train/", transform=trn_transform)
val_dataset = ImageDataset(val_df, "../../../data/raw/train/", transform=tst_transform)
tst_dataset = ImageDataset("../../../data/raw/sample_submission.csv",
                           "../../../data/raw/test/", transform=tst_transform)

# 4️⃣ Loader 정의
trn_loader = DataLoader(trn_dataset, batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=num_workers, pin_memory=True)
tst_loader = DataLoader(tst_dataset, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=num_workers, pin_memory=True)


In [8]:
from torch.utils.data import DataLoader

# Train loader
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=pin_memory,
    persistent_workers=persistent_workers,
    prefetch_factor=prefetch_factor,
    drop_last=True
)

# Validation loader
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=pin_memory,
    persistent_workers=persistent_workers,
    prefetch_factor=prefetch_factor
)

# Test loader (inference 전용)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=pin_memory,
    persistent_workers=persistent_workers,
    prefetch_factor=prefetch_factor
)


In [9]:
import torch
import torch.nn as nn
import timm

# ------------------------
# 🧠 Load Model
# ------------------------
model = timm.create_model(
    model_name,                 # "convnext_tiny.fb_in22k_ft_in1k"
    pretrained=True,
    num_classes=17,             # 문서 클래스 개수
    in_chans=3,
    drop_path_rate=0.1          # stochastic depth 살짝 추가 (일반화↑)
).to(device)

# ------------------------
# 💥 Loss Function
# ------------------------
# 문서 이미지 분류에서는 약간의 label noise 가능성 있으므로 Label Smoothing 추천
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)

# ------------------------
# ⚙️ Optimizer
# ------------------------
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LR,                      # 5e-4
    weight_decay=1e-4            # 일반적 weight decay (1e-2 → 1e-4로 안정화)
)

# ------------------------
# 📈 Scheduler
# ------------------------
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=EPOCHS,
    eta_min=1e-5                # 후반 학습 안정화
)

# ------------------------
# ⚡ AMP Scaler
# ------------------------
scaler = torch.cuda.amp.GradScaler()


/tmp/ipykernel_42617/997026691.py:43: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [10]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
import torch

# ------------------------
# ⚙️ Validation 함수 정의
# ------------------------
@torch.no_grad()
def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss, preds_list, targets_list = 0.0, [], []

    pbar = tqdm(loader, desc="Validating", leave=False)
    for images, targets in pbar:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        with torch.amp.autocast('cuda'):
            preds = model(images)
            loss = loss_fn(preds, targets)

        val_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_postfix(loss=f"{loss.item():.4f}")

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average="macro")

    return {
        "val_loss": val_loss,
        "val_acc": val_acc,
        "val_f1": val_f1,
    }


# ------------------------
# 🚀 Training Loop
# ------------------------
best_f1 = 0.0

for epoch in range(EPOCHS):
    # 1️⃣ Train for one epoch
    train_metrics = train_one_epoch(trn_loader, model, optimizer, loss_fn, device, scaler)

    # 2️⃣ Validation
    val_metrics = validate(val_loader, model, loss_fn, device)

    # 3️⃣ Scheduler step
    scheduler.step()

    # 4️⃣ Logging
    print(f"\n[Epoch {epoch+1}/{EPOCHS}]")
    print(f"Train | Loss: {train_metrics['train_loss']:.4f} | "
          f"Acc: {train_metrics['train_acc']:.4f} | "
          f"F1: {train_metrics['train_f1']:.4f}")
    print(f"Valid | Loss: {val_metrics['val_loss']:.4f} | "
          f"Acc: {val_metrics['val_acc']:.4f} | "
          f"F1: {val_metrics['val_f1']:.4f}")

    # 5️⃣ Save best model
    if val_metrics["val_f1"] > best_f1:
        best_f1 = val_metrics["val_f1"]
        torch.save(
            {
                "epoch": epoch + 1,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "scheduler_state_dict": scheduler.state_dict(),
                "val_loss": val_metrics["val_loss"],
                "val_f1": val_metrics["val_f1"],
            },
            f"./best_model_f1_{best_f1:.4f}_epoch{epoch+1}.pt"
        )
        print(f"✅ Best model saved! (F1: {best_f1:.4f})")


Training:   0%|                                                                                                                         | 0/19 [00:00<?, ?it/s]/tmp/ipykernel_42617/459546472.py:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                                                                                                                                                               


[Epoch 1/15]
Train | Loss: 2.1260 | Acc: 0.4013 | F1: 0.3604
Valid | Loss: 1.2944 | Acc: 0.6592 | F1: 0.5639
✅ Best model saved! (F1: 0.5639)


Training:   0%|                                                                                                                         | 0/19 [00:00<?, ?it/s]/tmp/ipykernel_42617/459546472.py:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                                                                                                                                                               


[Epoch 2/15]
Train | Loss: 1.2254 | Acc: 0.7434 | F1: 0.7181
Valid | Loss: 0.9446 | Acc: 0.8599 | F1: 0.8140
✅ Best model saved! (F1: 0.8140)


Training:   0%|                                                                                                                         | 0/19 [00:00<?, ?it/s]/tmp/ipykernel_42617/459546472.py:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                                                                                                                                                               

...


[Epoch 13/15]
Train | Loss: 0.6275 | Acc: 0.9885 | F1: 0.9882
Valid | Loss: 0.6684 | Acc: 0.9809 | F1: 0.9791


Training:   0%|                                                                                                                         | 0/19 [00:00<?, ?it/s]/tmp/ipykernel_42617/459546472.py:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                                                                                                                                                               


[Epoch 14/15]
Train | Loss: 0.6134 | Acc: 0.9918 | F1: 0.9916
Valid | Loss: 0.6650 | Acc: 0.9809 | F1: 0.9791


Training:   0%|                                                                                                                         | 0/19 [00:00<?, ?it/s]/tmp/ipykernel_42617/459546472.py:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                                                                                                                                                               


[Epoch 15/15]
Train | Loss: 0.6113 | Acc: 0.9926 | F1: 0.9919
Valid | Loss: 0.6649 | Acc: 0.9809 | F1: 0.9791


# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [11]:
from tqdm import tqdm
import pandas as pd
import torch

# ✅ 모델 저장 (훈련 완료 후, 추론 전에)
torch.save(model.state_dict(), "./final_model_convnext_tiny.pt")
print("✅ Model weights saved as final_model_convnext_tiny.pt")

# ✅ 추론 (Inference)
preds_list = []

model.eval()
for image, _ in tqdm(tst_loader, desc="Inference"):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)

    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

# ✅ 결과 DataFrame 생성
pred_df = pd.DataFrame(tst_dataset.df, columns=["ID", "target"])
pred_df["target"] = preds_list

# ✅ 샘플 제출 파일과 정렬 검증
sample_submission_df = pd.read_csv("../../../data/raw/sample_submission.csv")
assert (sample_submission_df["ID"] == pred_df["ID"]).all(), "⚠️ ID 순서 불일치!"

# ✅ 최종 제출 파일 저장
pred_df.to_csv("pred_3.csv", index=False)
print("🎯 Inference complete! Saved to pred_2.csv")


✅ Model weights saved as final_model_convnext_tiny.pt


Inference: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:12<00:00,  4.00it/s]

🎯 Inference complete! Saved to pred_2.csv
